In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [ ]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [55]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('patrol_dpr')

In [56]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [57]:
cols = list(col_rename.values())

### Read the current data from SQL

In [43]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_patrol'

In [44]:
patrol_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['patrol_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [45]:
sql_cols = list(patrol_sql.columns.values)

In [46]:
patrol_sql.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough


In [12]:
hash_rows(patrol_sql, exclude_cols = ['encounter_timestamp'], hash_name = 'row_hash')

### Add a section to try reading in the Site Reference list since it cannot go into a DB in the current state

In [33]:
col_rename = {'PROPERTY_I': 'site_id',
               'DESCRIPTIO': 'site_desc', 
               'DISTRICT': 'park_district', 
               'DESC_LOCAT': 'desc_location', 
               'Latitiude': 'latitude', 
               'Longitude': 'longitude'}

In [35]:
cols = list(col_rename.values())

In [37]:
sheet = client.open('DailyTasks_WebMerc_Centroids')

In [38]:
ws = sheet.worksheet('Sheet1')

In [39]:
site_ref = (get_as_dataframe(ws, evaluate_formulas = True, header= 0)
            .rename(columns = col_rename)
            .fillna(value = np.nan, axis = 1))[cols]

### Read the site reference list from SQL

In [26]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [ ]:
site_ref = pd.read_sql(con = engine, sql = sql)

### Read the latest data from Google Sheets

In [50]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [51]:
sheet = client.open('COMBINED Patrol Reporting Responses')

In [52]:
ws = sheet.worksheet('MASTER')

In [16]:
#patrol_hist = client.open_by_url('https://docs.google.com/spreadsheets/d/name/edit#gid=0/revisions')

In [17]:
#patrol_hist = (get_as_dataframe(hist.worksheet('MASTER'), evaluate_formulas = True, header= 0)
#               .rename(columns = col_rename))[list(col_rename.values())]

In [18]:
#patrol_hist = patrol_hist[patrol_hist['encounter_timestamp'].notna()]

In [58]:
#Read the worksheet as a data frame, rename the columns and subset the columns to only include those
#in the column list
patrol = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [59]:
patrol.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdassist,sd_pdcontact,sd_comments,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
1,5/21/2020 20:01:08,5/21/2020 20:00:00,Astoria Park,NaN,PEP,Domingo,Vigil,Alyssa,Gonzalez,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
2,5/21/2020 19:57:00,5/21/2020 19:30:00,Harlem River Park,Athletic Field,PEP,C.,Palmer,M.,Obioha,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
3,5/21/2020 19:52:22,5/21/2020 19:50:00,Riverside Park South,NaN,PEP,Arm,Zaman,Zuriel,Lara,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
4,5/21/2020 19:44:30,5/21/2020 19:29:00,Thomas Boyland Park,NaN,PEP,S,Johnson,L,LEE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn
5,5/21/2020 19:43:08,5/21/2020 19:42:00,Devoe Park,NaN,PEP,ivan,bernardez,Kevin,Feliciano,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx


In [60]:
yesno = ['closed_education', 'closed_outcome', 'closed_summonsissued', 'closed_pdassist',
         'closed_pdcontact', 'sd_summonsissued', 'sd_pdassist', 'sd_pdcontact']

In [61]:
yesno_cols(patrol, yesno)

In [62]:
#Remove any rows with no data, presumably these are rows with no timestamp
patrol = patrol[patrol['encounter_timestamp'].notna()]

In [63]:
patrol = patrol.merge(site_ref, how = 'left', on = 'site_desc')#[sql_cols]

In [64]:
patrol[patrol['site_id'].isnull()]['site_desc'].unique()

array(['Lower Highland Park',
       'Yellowstone Municipal Park-Katzman Playground',
       'Baisley Pond Park-155th St Playground',
       'Uncle Vito F. Maranzano Glendale Playground', 'St James Park',
       'Parade Place'], dtype=object)

In [25]:
patrol.to_sql('tbl_dpr_patrol3', engine, index = False, if_exists = 'append')

ProgrammingError: (pyodbc.ProgrammingError) ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'site_desc'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)")
[SQL: INSERT INTO tbl_dpr_patrol3 (encounter_timestamp, encounter_datetime, site_desc, location_adddesc, park_division, firstname_1, lastname_1, firstname_2, lastname_2, firstname_3, lastname_3, visit_reason, patrol_method, encounter_type, closed_amenity, closed_patroncount, closed_comments, sd_patronscomplied, sd_patronsnocomply, sd_amenity, sd_comments, summonscount_a01, summonscount_a03, summonscount_a04, summonscount_a22, other_summonstype, other_summonscount, borough, closed_education, closed_outcome, closed_summonsissued, closed_pdassist, closed_pdcontact, sd_summonsissued, sd_pdassist, sd_pdcontact) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[p

In [78]:
hash_rows(patrol, exclude_cols = ['encounter_timestamp'], hash_name = 'row_hash')

In [91]:
patrol_deltas = (check_deltas(new_df = patrol, old_df = patrol_sql, on = 'encounter_timestamp', 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))

In [92]:
patrol_inserts = patrol_deltas[patrol_deltas['dml_verb'] == 'I']

In [94]:
patrol_inserts.head()

,encounter_timestamp,encounter_datetime,property,location_adddesc,parks_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity_old,summonscount_a01_old,summonscount_a03_old,summonscount_a04_old,summonscount_a22_old,other_summonstype_old,other_summonscount_old,borough_old,row_hash_old,dml_verb
0,5/19/2020 20:37:55,5/19/2020 20:36:00,J.J. Byrne Playground,NaN,PEP,K,Romero,R,Anderson,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I
1,5/19/2020 20:34:58,5/19/2020 19:55:00,Inwood Hill Park - Dyckman Ballfield,NaN,PEP,Ariel,Junco,McTerry,Obioha,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I
2,5/19/2020 20:25:18,5/19/2020 18:30:00,Battery Park-Battery Park,NaN,PEP,daniel,palladino,a,mota,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I
3,5/19/2020 20:13:33,5/19/2020 20:12:00,Fort Greene Park,NaN,PEP,Daniel,Gitel,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I
4,5/19/2020 20:12:57,5/19/2020 20:15:00,Josephine Caminiti Playground,NaN,PEP,j,hengber,b,jospeh,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I


In [93]:
patrol_updates = patrol_deltas[patrol_deltas['dml_verb'] == 'U']

In [95]:
patrol_updates.head()

,encounter_timestamp,encounter_datetime,property,location_adddesc,parks_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity_old,summonscount_a01_old,summonscount_a03_old,summonscount_a04_old,summonscount_a22_old,other_summonstype_old,other_summonscount_old,borough_old,row_hash_old,dml_verb
